In [9]:
import tensorflow.keras.layers as lyrs
import tensorflow.keras.models as mod
import sklearn.preprocessing as prepro
import numpy
import matplotlib.pyplot as plt

In [10]:
def load_nasdaq():
    f = open('nasdaq100.csv', 'rb').readlines()[1:]
    raw_max = []
    raw_min = []
    raw_dates = []
    for b_line in f:
        line = b_line.decode("utf-8")
        min_price = float(line.split(',')[3])
        max_price = float(line.split(',')[2])
        raw_min.append(min_price)
        raw_max.append(max_price)
    return prepro.scale(raw_max), prepro.scale(raw_min)

In [11]:
# On importe le Dataset
raw_max, raw_min = load_nasdaq()
length = len(raw_max)
TRAIN_SIZE = int(length*0.8)

In [12]:
# On déclare les tableaux qui vont stoquer les valeurs du dataset
train_min = []
test_min = []
train_target_min = []
test_target_min = []

train_max = []
test_max = []
train_target_max = []
test_target_max = []

In [13]:
# On stoque les données de train et de target
# Train = Stock market des 30 derniers jours
# Target = Stock market du lendemain

for i in range(31, length):
    if i < TRAIN_SIZE:
        train_min.append(raw_min[i-31:i-1])
        train_max.append(raw_max[i-31:i-1])
        train_target_min.append(1 if raw_min[i] > raw_min[i-1] else 0)
        train_target_max.append(1 if raw_max[i] > raw_max[i-1] else 0)
    else:
        test_min.append(raw_min[i-31:i-1])
        test_max.append(raw_max[i-31:i-1])
        test_target_min.append(1 if raw_min[i] > raw_min[i-1] else 0)
        test_target_max.append(1 if raw_max[i] > raw_max[i-1] else 0)

In [14]:
# Conversion en numpy pour des manipulations futures

train_min = numpy.array(train_min)
train_target_min = numpy.array(train_target_min)
test_min = numpy.array(test_min)
test_target_min = numpy.array(test_target_min)

train_max = numpy.array(train_max)
train_target_max = numpy.array(train_target_max)
test_max = numpy.array(test_max)
test_target_max = numpy.array(test_target_max)

In [15]:
# Modèle de classification du min
model_min = mod.Sequential()
model_min.add(lyrs.Dense(500, input_shape = (30,)))
model_min.add(lyrs.Activation('relu'))
model_min.add(lyrs.Dropout(0.25))
model_min.add(lyrs.Dense(250))
model_min.add(lyrs.Activation('relu'))
model_min.add(lyrs.Dense(1))
model_min.add(lyrs.Activation('softmax'))
model_min.compile(optimizer="adam", loss='binary_crossentropy', metrics=['accuracy'])

In [17]:
# Modèle de classification du max
model_max = mod.Sequential()
model_max.add(lyrs.Dense(500, input_shape = (30,)))
model_max.add(lyrs.Activation('relu'))
model_max.add(lyrs.Dropout(0.25))
model_max.add(lyrs.Dense(250))
model_max.add(lyrs.Activation('relu'))
model_max.add(lyrs.Dense(1))
model_max.add(lyrs.Activation('softmax'))
model_max.compile(optimizer="adam", loss='binary_crossentropy', metrics=['accuracy'])

In [18]:
history_min = model_min.fit(train_min, 
          train_target_min, 
          epochs=6,
          batch_size = 10, 
          verbose=1, 
          validation_data=(test_min, test_target_min))
score_min = model_min.evaluate(train_min, train_target_min, batch_size=10)
print(score_min)
predicted_min = model_min.predict(test_min)

Epoch 1/6
98/98 [==============================] - 0s 3ms/step - loss: 6.4372 - accuracy: 0.5779 - val_loss: 5.9908 - val_accuracy: 0.6071
Epoch 2/6
98/98 [==============================] - 0s 1ms/step - loss: 6.4372 - accuracy: 0.5779 - val_loss: 5.9908 - val_accuracy: 0.6071
Epoch 3/6
98/98 [==============================] - 0s 1ms/step - loss: 6.4372 - accuracy: 0.5779 - val_loss: 5.9908 - val_accuracy: 0.6071
Epoch 4/6
98/98 [==============================] - 0s 1ms/step - loss: 6.4372 - accuracy: 0.5779 - val_loss: 5.9908 - val_accuracy: 0.6071
Epoch 5/6
98/98 [==============================] - 0s 2ms/step - loss: 6.4372 - accuracy: 0.5779 - val_loss: 5.9908 - val_accuracy: 0.6071
Epoch 6/6
98/98 [==============================] - 0s 694us/step - loss: 6.4372 - accuracy: 0.5779
[6.437180042266846, 0.5778688788414001]


In [19]:
history_max = model_min.fit(train_max, 
          train_target_max, 
          epochs=6,
          batch_size = 10, 
          verbose=1, 
          validation_data=(test_max, test_target_max))
score_max = model_min.evaluate(train_max, train_target_max, batch_size=10)
print(score_max)
predicted_max = model_min.predict(test_max)

Epoch 1/6
98/98 [==============================] - 0s 2ms/step - loss: 6.6090 - accuracy: 0.5666 - val_loss: 5.8697 - val_accuracy: 0.6151
Epoch 2/6
98/98 [==============================] - 0s 1ms/step - loss: 6.6090 - accuracy: 0.5666 - val_loss: 5.8697 - val_accuracy: 0.6151
Epoch 3/6
98/98 [==============================] - 0s 1ms/step - loss: 6.6090 - accuracy: 0.5666 - val_loss: 5.8697 - val_accuracy: 0.6151
Epoch 4/6
98/98 [==============================] - 0s 1ms/step - loss: 6.6090 - accuracy: 0.5666 - val_loss: 5.8697 - val_accuracy: 0.6151
Epoch 5/6
98/98 [==============================] - 0s 1ms/step - loss: 6.6090 - accuracy: 0.5666 - val_loss: 5.8697 - val_accuracy: 0.6151
Epoch 6/6
98/98 [==============================] - 0s 658us/step - loss: 6.6090 - accuracy: 0.5666
[6.60904598236084, 0.5665983557701111]


In [20]:
print(predicted_max)
print(predicted_min)

[[1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.

In [ ]:
# Le soucis avec ce réseau est qu'en prédisant sur une période aléatoire que le marché va monter 
# - Sans prendre en considération le marché des autres jours -
# On aurait raison 61% du temps car le marché est à la hausse
# Pour la troisième approche, nous allons tenter de l'apprentissage non-supervisé
# Sur un marché plus constant